In [ ]:
# export
import json,fire,re,os,shutil,glob
from pathlib import Path
from fastai.gen_doc.nbdoc import show_doc
from typing import Union, Optional
from typeguard import typechecked
NoneType = type(None)

In [ ]:
# default_exp export

## Type checking

Runtime type checking is handy, so let's make it easy!

In [ ]:
#export
def chk(f): return typechecked(always=True)(f)

In [ ]:
@chk
def test_chk(a:Union[int,NoneType]=None): pass
test_chk(1)
test_chk()

## Reading a notebook

### Finding export cells

A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries.

In [ ]:
test_nb = json.load(open('00_export.ipynb'))

The root has four keys: `cells` contains the cells of the notebook, `metadata` some stuff around the version of python used to execute the notebook, `nbformat` and `nbformat_minor` the version of nbformat. 

In [ ]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [ ]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.1'}}

In [ ]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.2'

The cells key then contains a list of cells. Each one is a new dictionary that contains entries like the type (code or markdown), the source (what is written in the cell) and the output (for code cells).

In [ ]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {},
 'outputs': [],
 'source': ['# export\n',
  'import json,fire,re,os,shutil\n',
  'from pathlib import Path']}

This function will be used to detect different `# code` at the beginning of cells.

In [ ]:
# export
def check_re_pattern(cell, pat):
    "Check if `cell` contains given `pat`."
    if cell['cell_type'] != 'code': return False
    src = cell['source']
    if len(src) == 0: return False
    return re.match(pat, src[0], re.IGNORECASE) 

Now we can check for the cell that need to be exported.

In [ ]:
# export
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module."
    if check_re_pattern(cell, r'^\s*#\s*exports?\s*$'): return default
    tst = check_re_pattern(cell, r'^\s*#\s*exports?\s*(\S+)\s*$')
    return os.path.sep.join(tst.groups()[0].split('.')) if tst else None 

We check if the cell begins with an `#export` or `#exports` code, potentially with a module name where we want it exported. The default is given in a cell of the form `#default_exp bla` inside the notebook (usually at the top), though in this function, it needs the be passed (the final script will read the whole notebook to find it).

In [ ]:
cell = test_nb['cells'][0].copy()
assert is_export(cell, 'export') == 'export'
cell['source'][0] = "# exports" 
assert is_export(cell, 'export') == 'export'
cell['source'][0] = "# export mod" 
assert is_export(cell, 'export') == 'mod'
cell['source'][0] = "# export mod.file" 
assert is_export(cell, 'export') == 'mod/file'
cell['source'][0] = "# expt mod.file"
assert is_export(cell, 'export') is None
cell['source'][0] = "# export"

### Finding the default export path

In [ ]:
# export
def find_default_export(cells):
    for cell in cells:
        tst = check_re_pattern(cell, r'^\s*#\s*default_exp\s*(\S*)\s*$')
        if tst: return tst.groups()[0]

In [ ]:
find_default_export(test_nb['cells'])

'export'

### Exporting one notebook

In [ ]:
#export 
def _notebook2script(fname):
    "Finds cells starting with `#export` and puts them into a new module"
    fname = Path(fname)
    nb = json.load(open(fname,'r'))
    default = find_default_export(nb['cells'])
    if default is None: 
        print(f"Skipping conversion of {fname}: please add an #default_exp cell.")
        return
    default = os.path.sep.join(default.split('1.'))
    fname_out = Path.cwd()/'fastai_local'/f'{default}.py'
    code_cells = [c for c in nb['cells'] if is_export(c, default) is not None]
    module = f'''
#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev/{fname.name}

'''
    for cell in code_cells: module += ''.join(cell['source'][1:]) + '\n\n'
    # remove trailing spaces
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)
    output_path = fname.parent/'exp'/fname_out
    open(output_path,'w').write(module[:-2])
    print(f"Converted {fname} to {output_path}")

### Exporting more notebooks

In [ ]:
#export 
def _get_sorted_files(all_fs: Union[bool,str], up_to=None):
    "Return the list of files corresponding to `g` in the current dir."
    if (all_fs==True): ret = glob.glob('*.ipynb') # Checks both that is bool type and that is True
    else: ret = glob.glob(all_fs) if isinstance(g,str) else []
    if len(ret)==0: print('WARNING: No files found')
    if up_to is not None: ret = [f for f in ret if str(f)<=str(up_to)]
    return sorted(ret)

Note that `g` can either be `True` (for all notebook files in the directory) or a glob pattern (like `00_*.ipynb`). If `up_to` is passed, notebooks are only converted up to that point (since our notebooks are beginning with numbers).

In [ ]:
#export 
def notebook2script(fname=None, all_fs:Optional[Union[bool,str]]=None, up_to=None):
    # initial checks
    assert fname or all_fs
    if (all_fs is None) and (up_to is not None): all_fs=True # Enable allFiles if upTo is present
    fnames = _get_sorted_files(all_fs, up_to=up_to) if all_fs else [fname]
    [_notebook2script(f) for f in fnames]

Finds cells starting with `#export` and puts them into a new module.
* `fname`: the filename of one notebook to convert
* `all_fs`: `True` if you want to convert all notebook files in the folder or a glob expression
* `up_to`: converts all notebooks respecting the previous arg up to a certain number

Examples of use:
```
notebook2script 00_export.ipynb                 # Parse 00_export.ipynb
notebook2script --all_fs=True                   # Parse all files
notebook2script --all_fs=nb*                    # Parse all files starting with nb*
notebook2script --up_to=10                      # Parse all files with (name<='10')
notebook2script --all_fs=*_*.ipynb --up_to=10   # Parse all files with an '_' and (name<='10')
```

In [ ]:
notebook2script('00_export.ipynb')

Converted 00_export.ipynb to /home/ubuntu/fastai_docs/dev/fastai_local/export.py


In [ ]:
notebook2script(all_fs=True)

Converted 00_export.ipynb to /home/ubuntu/fastai_docs/dev/fastai_local/export.py
Converted 01_test.ipynb to /home/ubuntu/fastai_docs/dev/fastai_local/test.py
Skipping conversion of 03_core.ipynb: please add an #default_exp cell.
